# CS295/395: Secure Distributed Computation
## Homework 10

In [1]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request
import hashlib
from nacl.signing import SigningKey

The structure below implements simplified transactions in the style of Bitcoin. Reference [this page](https://en.bitcoin.it/wiki/Transaction) for more information about the structure of transactions.

In [21]:
HashPointer = namedtuple('HashPointer', ['hash', 'pointer'])
Transaction = namedtuple('Transaction', ['inputs', 'outputs'])

Input = namedtuple('Input', ['previous_tx', 'index', 'public_key'])
SignedInput = namedtuple('SignedInput', ['input', 'signed_input'])

Output = namedtuple('Output', ['public_key_hash', 'value'])

DIFFICULTY = int(2**(32 * 8)/1000)

class Block:
    def __init__(self, transactions, prev, nonce, pubkey_hash):
        self.transactions = transactions
        self.prev = prev
        self.nonce = nonce
        self.pubkey_hash = pubkey_hash
    
    def __repr__(self):
        return f'\nBlock(\n transaction: {self.transactions},\n nonce: {self.nonce},\n prev: {self.prev})'

def bytes_value(v):
    return bytes(str(v), encoding='utf-8')

def hash_value(v):
    return hashlib.sha256(bytes_value(v)).hexdigest()

## Question 1 (20 points)

Implement `mine_for_block`, a function to add a block to the blockchain. It should ensure that the hash of the blockchain is less than or equal to `DIFFICULTY`.

In [23]:
def add_block(transactions, blockchain, nonce, pubkey_hash):
    prev_hash = hash_value(blockchain)
    prev = HashPointer(prev_hash, blockchain)
    new_block = Block(transactions, prev, nonce, pubkey_hash)
    
    return new_block, hash_value(new_block)

def mine_for_block(transactions, pubkey_hash, blockchain):
    nonce = 0
    while True:
        prev_hash = hash_value(blockchain)
        prev = HashPointer(prev_hash, blockchain)
        new_block = Block(transactions, prev, nonce, pubkey_hash)
        hash = int(hash_value(new_block), 16)
        if hash < DIFFICULTY:
            print(f'Found nonce: {nonce}')
            return add_block(transactions, blockchain, nonce, pubkey_hash)
        nonce += 1
    

In [24]:
# TEST CASE
print(1)
key_bob = SigningKey.generate()
print(2)
key_alice = SigningKey.generate()
print(3)
pubkey_bob = key_bob.verify_key
pubkey_alice = key_alice.verify_key

print(4)

i1 = Input('COINBASE', 0, pubkey_bob)
i1_signed = SignedInput(i1, key_bob.sign(bytes_value(i1)))
o1 = Output(hash_value(pubkey_bob), 10.0)
tx1 = Transaction([i1_signed], [o1])

print(5)

i2 = Input(hash_value(tx1), 0, pubkey_bob)
i2_signed = SignedInput(i2, key_bob.sign(bytes_value(i2)))

print(6)

o2 = Output(hash_value(pubkey_bob), 9.0)
o3 = Output(hash_value(pubkey_alice), 1.0)
tx2 = Transaction([i2_signed], [o2, o3])

print(7)

b1, b1_hash = mine_for_block([tx2, tx1], hash_value(pubkey_bob), None)



1
2
3
4
5
6
7
Found nonce: 288


In [25]:
# Bob's going to mine some coins



# And pay some to Alice


# Mine the block


# Alice pays some money back to Bob
i3 = Input(hash_value(tx2), 1, pubkey_alice)
i3_signed = SignedInput(i3, key_alice.sign(bytes_value(i3)))
o4 = Output(hash_value(pubkey_alice), 0.5)
o5 = Output(hash_value(pubkey_bob), 0.5)
tx3 = Transaction([i3_signed], [o4, o5])
b2, b2_hash = mine_for_block([tx3], hash_value(pubkey_bob), b1)

Found nonce: 26


## Question 2 (20 points)

Implement `validate_blockchain`, a function to validate the blockchain. It should check that:

1. The structure of the blockchain is valid
2. Each block satisfies the difficulty requirement
3. Each transaction is valid

The function should throw an error if the blockchain is not valid, and return `True` if it is valid.

In [54]:
def check_blockchain(blockchain: Block, expected_hash):
    # make sure blockchain hashes to expected hash
    assert hash_value(blockchain) == expected_hash

    # make sure hash value satisfies difficulty requirement
    assert int(hash_value(blockchain), 16) < DIFFICULTY or blockchain == None

    for transaction in blockchain.transactions:
        signed_inputs, outputs = transaction
        for s_input in signed_inputs:
            input, signature = s_input
            prev_tx, index, pub_key = input
            assert pub_key.verify(bytes_value(input), signature.signature)

        
    hash, pointer = blockchain.prev

    if pointer == None:
        return True
    else:
        return check_blockchain(pointer, hash)

In [55]:
# TEST CASE
assert check_blockchain(b2, b2_hash)

## Question 3 (20 points)

In 3-5 sentences, describe what would be required to modify this scheme to support *proof-of-stake mining*, in the style of Peercoin. Reference Section 8.5 in [the textbook](https://d28rh4a8wq0iu5.cloudfront.net/bitcointech/readings/princeton_bitcoin_book.pdf).

The goal is to decrease the difficulty for miners who have "staked" some coins. Normally the issue of "pretending to be two people to get two chances at the reward" is prevented by requiring twice the computing power. Now it is prevented by requiring you to have twice the funds. There would be a new type of transaction, a "staking" transaction, and then while you mine, each person has a custom difficulty that is inversely proportionaly to the size of their stake. This means that the blockchain lies in the hands of the large stake holders as opposed to the large compute holders, and the well being incentive is baked in because if you hold a large amount of the coin, you want that coin to succeed.